In [12]:
import csv
# python 读取文件
with open("/root/yeast_data/data/RAN_seq/SraRunInfo.csv") as f:
    # python 读取csv
    reader = csv.reader(f)
    print(reader)

In [34]:
import pandas as pd
df = pd.read_csv("/root/yeast_data/data/RAN_seq/SraRunInfo.csv")
df

,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR1916152,2015-04-03 17:06:06,2015-03-15 15:16:45,17028551,868456101,0,51,885,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,GEO,SRA246757,NaN,public,AF738628B3B21AF0B2D5222743D803C2,608F2BCDA164CC23D1178526A9B1DD71
1,SRR1916153,2015-04-03 17:06:06,2015-03-15 15:16:51,16542825,843684075,0,51,862,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,GEO,SRA246757,NaN,public,354CD8BF3729EE6F6ED491F76DECC476,2C9F3C341FFDD3869E3740A6E75CF7AF
2,SRR1916154,2015-04-03 17:06:06,2015-03-15 15:16:39,16739897,853734747,0,51,874,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,GEO,SRA246757,NaN,public,FE3A847B05887981D5F764D58F3138C4,954C10D5258D83193BCE73CD256D90E8
3,SRR1916155,2015-04-03 17:06:06,2015-03-15 15:16:33,16534790,843274290,0,51,862,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,GEO,SRA246757,NaN,public,CC44A1715233D6FE50FCF1B9CF8FDE14,5EA08B9FDB5359DCB4E25162EE8A12AC
4,SRR1916156,2015-04-03 17:06:06,2015-03-15 15:16:39,15793246,805455546,0,51,827,NaN,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,GEO,SRA246757,NaN,public,396A87E0D6BDE9A0F5AAA11C0D6D95BB,00A7ADB71C6CF1654686C60F425DB897


In [71]:
for index,row in df.iterrows():
    cmd_str = "fastq-dump {0} \nfastqc {0}.fastq".format(row[0])
    print(cmd_str)

fastq-dump SRR1916152 
fastqc SRR1916152.fastq
fastq-dump SRR1916153 
fastqc SRR1916153.fastq
fastq-dump SRR1916154 
fastqc SRR1916154.fastq
fastq-dump SRR1916155 
fastqc SRR1916155.fastq
fastq-dump SRR1916156 
fastqc SRR1916156.fastq


### hisat2将reads比对到参考基因组

In [2]:
with open("/root/yeast_data/data/RAN_seq/SraAccList.txt") as acc_file_handle:
    for line in acc_file_handle:
        line = line.strip()                                                       
        line = line.split("\t")
        cmd_str="hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/{0}.fastq | samtools view -bS - | samtools sort - -o {1}.bam".format(line[0],line[1])
        #cmd_str+="\nsamtools index {1}.bam
        print(cmd_str)

hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916152.fastq | samtools view -bS - | samtools sort - -o EV_3.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916153.fastq | samtools view -bS - | samtools sort - -o EV_4.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916154.fastq | samtools view -bS - | samtools sort - -o DNMT3B_2.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916155.fastq | samtools view -bS - | samtools sort - -o DNMT3B_3.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916156.fastq | samtools view -bS - | samtools sort - -o DNMT3B_4.bam


### read count


In [4]:
with open("/root/yeast_data/data/RAN_seq/SraAccList.txt") as acc_file_handle:
    for line in acc_file_handle:
        line = line.strip()                                                       
        line = line.split("\t")
        cmd_str="htseq-count -f bam -r pos ../read_aln/{1}.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > {1}.count.tab".format(line[0],line[1])
        #cmd_str+="\nsamtools index {1}.bam
        
        print(cmd_str)



htseq-count -f bam -r pos ../read_aln/EV_3.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > EV_3.count.tab
htseq-count -f bam -r pos ../read_aln/EV_4.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > EV_4.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_2.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_2.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_3.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_3.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_4.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_4.count.tab


In [5]:
import os
base_dir = "/root/yeast_data/analysis/htseq-count"
dir_list = os.listdir(base_dir)
file_name_list = []
for i in dir_list:
    if os.path.splitext(i)[1] == '.tab':
        file_name_list.append(i)

print(file_name_list)
count_dict = {}
for file_name in file_name_list:
    with open(base_dir+"/"+file_name) as f:
        for line in f:
            if line.startswith("__"):
                continue
            line = line.rstrip("\n")
            ele = line.split()
            if ele[0] in count_dict.keys():
                count_dict.get(ele[0]).append(ele[1])
            else:
                count_dict[ele[0]]=[ele[1]]
# print(count_dict)
result_str=""
#print("gene_id\t" + "\t".join([x[:-10] for x in file_name_list]))
result_str +="gene_id\t" + "\t".join([x[:-10] for x in file_name_list])+"\n"
for gene_id in count_dict:
    #print(gene_id+"\t"+"\t".join(count_dict[gene_id]))
    result_str+= gene_id+"\t"+"\t".join(count_dict[gene_id])+"\n"

print(result_str[:200])
with open(base_dir+"/result.txt","w") as f:
    f.write(result_str)

['DNMT3B_2.count.tab', 'EV_3.count.tab', 'DNMT3B_3.count.tab', 'DNMT3B_4.count.tab', 'EV_4.count.tab']
gene_id	DNMT3B_2	EV_3	DNMT3B_3	DNMT3B_4	EV_4
ETS1-1	8	14	4	7	8
ETS1-2	0	0	0	0	0
ETS2-1	0	0	0	1	0
ETS2-2	0	0	0	0	0
HRA1	19	71	24	14	30
ICR1	203	142	210	188	159
IRT1	618	387	555	491	260
ITS1-1	7	17	4	6	
